1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [73]:
import json

# Load the JSON data from the file
with open('../../subdir/results/scenario1/score_alpha05_nodes50/blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}
transitions = []
nodes_by_block=[]
# transition = {"obs" : , "obs_" , "action" : , "reward" : ,"done" : }
# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id and transaction2['Content'].get('type') == 0:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId

# Print or use node_info_by_block as needed
for block_id, nodes in node_info_by_block.items():
    print(f"Block {block_id}:")
    nodes_by_block.append(nodes)
    for node_info in nodes:
        print(node_info)
   


Block 1/0:
[0, 1, 0.0, 400, 110.0, 170.0, 0.0, 0.0]
[1, 1, 0.0, 870, 130.0, 770.0, 0.0, 0.0]
[2, 1, 0.0, 910, 60.0, 710.0, 0.0, 0.0]
[3, 1, 0.0, 160, 150.0, 240.0, 0.0, 0.0]
[4, 1, 0.0, 930, 270.0, 240.0, 0.0, 0.0]
[5, 1, 0.0, 730, 90.0, 380.0, 0.0, 0.0]
[6, 1, 0.0, 780, 50.0, 700.0, 0.0, 0.0]
[7, 1, 0.0, 710, 90.0, 740.0, 0.0, 0.0]
[8, 0, 0.0, 660, 300.0, 730.0, 0.0, 0.0]
[9, 1, 0.0, 500, 300.0, 730.0, 0.0, 0.0]
[10, 1, 0.0, 230, 50.0, 780.0, 0.0, 32.61]
[11, 0, 0.0, 270, 230.0, 770.0, 0.0, 0.0]
[12, 1, 0.0, 460, 80.0, 220.0, 0.0, 0.0]
[13, 1, 0.0, 650, 210.0, 520.0, 0.0, 65.38]
[14, 1, 0.0, 810, 190.0, 600.0, 0.0, 0.0]
[15, 1, 0.0, 850, 100.0, 260.0, 0.0, 0.0]
[16, 1, 0.0, 290, 210.0, 960.0, 0.0, 0.0]
[17, 1, 0.0, 760, 110.0, 180.0, 0.0, 0.0]
[18, 0, 0.0, 570, 210.0, 180.0, 0.0, 0.0]
[19, 1, 0.0, 500, 80.0, 580.0, 0.0, 0.0]
[20, 1, 0.0, 390, 100.0, 260.0, 0.0, 0.0]
[21, 1, 0.0, 390, 160.0, 520.0, 0.0, 56.41]
[22, 1, 0.0, 660, 230.0, 730.0, 0.0, 62.88]
[23, 0, 0.0, 140, 150.0, 940.0, 

2 - get the global accuracy for each round

In [74]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [56.15, 69.66, 74.83, 77.24, 75.86, 81.38, 81.72, 83.1, 85.86, 87.14, 88.57, 87.14, 87.93, 90.62, 90.62, 86.43, 90.0, 90.0, 88.57, 88.57, 87.14, 92.74, 89.66, 92.62, 93.21, 91.82, 91.45, 90.91, 92.36]


In [75]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [34, 33, 16, 9, 43, 35, 45, 49, 48, 22, 47, 21, 24, 25, 13, 36, 10]
Block 1/1: [16, 9, 34, 43, 35, 45, 49, 48, 47, 8, 21, 24, 25, 13, 36, 10, 28]
Block 1/2: [16, 34, 35, 45, 33, 49, 23, 48, 11, 47, 8, 24, 25, 22, 13, 36, 10]
Block 1/3: [16, 34, 35, 45, 33, 49, 23, 48, 11, 47, 24, 22, 13, 36, 21, 28, 29]
Block 1/4: [16, 34, 45, 35, 33, 49, 23, 43, 48, 47, 24, 22, 13, 36, 21, 25, 10]
Block 1/5: [16, 34, 33, 45, 49, 23, 43, 48, 47, 24, 13, 36, 25, 31, 46, 3, 1]
Block 1/6: [16, 33, 34, 49, 23, 35, 43, 47, 24, 13, 36, 22, 21, 28, 29, 46, 14]
Block 1/7: [16, 33, 23, 45, 35, 43, 34, 47, 48, 24, 13, 36, 22, 21, 25, 10, 46]
Block 1/8: [16, 33, 23, 35, 49, 43, 34, 13, 36, 22, 25, 46, 29, 14, 37, 19, 44]
Block 1/9: [23, 49, 35, 43, 45, 36, 48, 34, 24, 22, 25, 10, 46, 14, 19, 44, 38]
Block 1/10: [23, 33, 35, 16, 43, 49, 36, 48, 22, 47, 21, 10, 46, 14, 31, 19, 44]
Block 1/11: [23, 33, 16, 35, 43, 49, 45, 36, 47, 24, 25, 10, 46, 14, 31, 19, 44]
Block 1/12: [16, 33, 23, 49, 45, 36, 48, 47,

In [76]:
round_reward = []
transitions = []

for block_id in range(len(node_info_by_block) - 1):  # Fixed the range
    # print(len(node_info_by_block) - 1)
    # print(f"Block {block_id}:")
    nodes_rewards = []
    
    for node_index in selected_nodes[block_id]:
        # print("Node index: ", node_index)
        node_data = nodes_by_block[block_id][node_index]
        reward = node_data[-1]
        # print("Original reward: ", reward)
        
        if reward != 0.0:
            reward *= node_data[2]
        else :
            reward = node_data[2]
        # print("Modified reward: ", reward)
        
        nodes_rewards.append(reward)
    
    # print("Nodes rewards:", nodes_rewards)
    cumulated_reward = sum(nodes_rewards)
    round_reward.append(cumulated_reward)
    
    transition = {
        "obs": nodes_by_block[block_id],
        "obs_": nodes_by_block[block_id + 1],
        "action": selected_nodes[block_id],
        "reward": cumulated_reward,
        "done": block_id == len(node_info_by_block) - 1  # Corrected the condition
    }
    transitions.append(transition)

for transition in transitions : 
    print(transition)


{'obs': [[0, 1, 0.0, 400, 110.0, 170.0, 0.0, 0.0], [1, 1, 0.0, 870, 130.0, 770.0, 0.0, 0.0], [2, 1, 0.0, 910, 60.0, 710.0, 0.0, 0.0], [3, 1, 0.0, 160, 150.0, 240.0, 0.0, 0.0], [4, 1, 0.0, 930, 270.0, 240.0, 0.0, 0.0], [5, 1, 0.0, 730, 90.0, 380.0, 0.0, 0.0], [6, 1, 0.0, 780, 50.0, 700.0, 0.0, 0.0], [7, 1, 0.0, 710, 90.0, 740.0, 0.0, 0.0], [8, 0, 0.0, 660, 300.0, 730.0, 0.0, 0.0], [9, 1, 0.0, 500, 300.0, 730.0, 0.0, 0.0], [10, 1, 0.0, 230, 50.0, 780.0, 0.0, 32.61], [11, 0, 0.0, 270, 230.0, 770.0, 0.0, 0.0], [12, 1, 0.0, 460, 80.0, 220.0, 0.0, 0.0], [13, 1, 0.0, 650, 210.0, 520.0, 0.0, 65.38], [14, 1, 0.0, 810, 190.0, 600.0, 0.0, 0.0], [15, 1, 0.0, 850, 100.0, 260.0, 0.0, 0.0], [16, 1, 0.0, 290, 210.0, 960.0, 0.0, 0.0], [17, 1, 0.0, 760, 110.0, 180.0, 0.0, 0.0], [18, 0, 0.0, 570, 210.0, 180.0, 0.0, 0.0], [19, 1, 0.0, 500, 80.0, 580.0, 0.0, 0.0], [20, 1, 0.0, 390, 100.0, 260.0, 0.0, 0.0], [21, 1, 0.0, 390, 160.0, 520.0, 0.0, 56.41], [22, 1, 0.0, 660, 230.0, 730.0, 0.0, 62.88], [23, 0, 0.0

In [77]:
import sys
sys.path.insert(0, '../')  # Add the directory containing "DRL" to the path

from agent import node_selection_agent as ag
from environment import node_selection_env as nds

total_nodes = 50
select_Agg = 7
select_trainer = 10
num_features = 8
target_accuracy = 99
max_rounds= 30
#create the nodes selection environment accoding to the observation
envNodeSelect = nds.FLNodeSelectionEnv(total_nodes,select_Agg,select_trainer, num_features=num_features, target=target_accuracy, max_rounds=max_rounds)
obs_shape = envNodeSelect.observation_space.sample()["current_state"].shape
print(obs_shape)
agent = ag.Agent(input_shape=obs_shape ,n_actions=obs_shape[0], env=envNodeSelect,max_actions=select_trainer+select_Agg) 
agent.load_models()

# train the agent with the transitions store the trasitions then make the agent learn
for transition in transitions : 
    agent.remember(transition["obs"],transition["action"], transition["reward"], transition["obs_"], transition["done"])
    agent.learn()

agent.save_models()


(50, 8)
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...


...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoin

In [78]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 34 was selected 10 times.
Node 33 was selected 24 times.
Node 16 was selected 26 times.
Node 9 was selected 2 times.
Node 43 was selected 25 times.
Node 35 was selected 25 times.
Node 45 was selected 22 times.
Node 49 was selected 28 times.
Node 48 was selected 21 times.
Node 22 was selected 23 times.
Node 47 was selected 23 times.
Node 21 was selected 21 times.
Node 24 was selected 25 times.
Node 25 was selected 21 times.
Node 13 was selected 9 times.
Node 36 was selected 26 times.
Node 10 was selected 23 times.
Node 8 was selected 2 times.
Node 28 was selected 3 times.
Node 23 was selected 25 times.
Node 11 was selected 2 times.
Node 29 was selected 3 times.
Node 31 was selected 19 times.
Node 46 was selected 20 times.
Node 3 was selected 1 times.
Node 1 was selected 1 times.
Node 14 was selected 14 times.
Node 37 was selected 15 times.
Node 19 was selected 20 times.
Node 44 was selected 18 times.
Node 38 was selected 8 times.
Node 7 was selected 1 times.
Node 42 was selected 3 